In [1]:
from utils import get_sample_config
orp, recp, sample_rate, min5, item = get_sample_config()

# ppg, ecg
from dataset import get_vital
ppg_or, ecg_or   = get_vital(orp, type_='or',sample_rate=sample_rate, duration=min5)
ppg_rec, ecg_rec = get_vital(recp, type_='rec',sample_rate=sample_rate, duration=min5)

In [2]:
from preprocess import list2specgram
x = list2specgram([ppg_or, ppg_rec, ecg_or, ecg_rec],type_=['ppg','ppg','ecg','ecg'], sample_rate=sample_rate)
ppg_or, ppg_rec, ecg_or, ecg_rec = x

In [77]:
import os
import pandas as pd
import json 

import vitaldb
import torch
from torch.nn import functional as F

from preprocess import list2specgram
from dataset import get_vital 

class VitalDataset(torch.utils.data.Dataset):
    def __init__(self,root_dir='../data/pd_gy',ann_path='../data/pd_gy/train.json',sample_rate=300,duration=300*60*5):
        
        with open(ann_path,'r') as f:
            jf = json.load(f)
        
        self.jf = jf
        self.root_dir = root_dir
        self.sample_rate = sample_rate
        self.duration = duration
        
    def __getitem__(self,idx):
        
        item = self.jf[idx]
        orp, recp = self.get_path(item) 

        ppg_or, ecg_or   = get_vital(orp,  type_='or' ,sample_rate=self.sample_rate,duration=self.duration)
        ppg_rec, ecg_rec = get_vital(recp, type_='rec',sample_rate=self.sample_rate,duration=self.duration)
        
        ppgecg = list2specgram([ppg_or, ppg_rec, ecg_or, ecg_rec],type_=['ppg','ppg','ecg','ecg'], sample_rate=self.sample_rate)
        ppg_or, ppg_rec, ecg_or, ecg_rec = ppgecg
        
        vital = np.array([ppg_or,ecg_or,ppg_rec,ecg_rec])
        vital = torch.tensor(vital)
        
        
        y = item['nrs_2']
        y = torch.tensor(y)
        y = F.one_hot(y,num_classes=2)
        
        return vital, y
    
    def __len__(self):
        return len(self.jf)
    
    def get_path(self, item):
        
        orp = item['or_path'][0].replace('\\','/')
        orp = os.path.join(self.root_dir, orp)
        recp = item['rec_path'][0].replace('\\','/')
        recp = os.path.join(self.root_dir,recp)
        return orp, recp
#     def get_vital(self,path,type_='or',sample_rate=300, duration=300*6*5):

#         vf = vitaldb.VitalFile(path,track_names=['Intellivue/PLETH','Intellivue/NIBP_SYS','Intellivue/ECG_II'])
#         ppg = vf.to_numpy('Intellivue/PLETH',1/sample_rate)
#         ecg = vf.to_numpy('Intellivue/ECG_II',1/sample_rate)
#         first_idx = 0 

#         if type_ == 'or': # starts from nibp
#             nibp_or = vf.to_numpy('Intellivue/NIBP_SYS',1/sample_rate)
#             for i, value in enumerate(nibp_or):
#                 if pd.isna(value):
#                     continue
#                 else:
#                     first_idx = i
#                     break

#         elif type_ == 'rec': # starts from non-nan
#             for i, value in enumerate(ppg):
#                 if pd.isna(value):
#                     continue
#                 else:
#                     first_idx = i
#                     break
        # ecg = ecg[first_idx:first_idx+duration]            
        # ppg = ppg[first_idx:first_idx+duration]            
        # return ppg, ecg


In [1]:
import torch
from torch.utils.data import DataLoader
from dataset import VitalDataset
trdt  = VitalDataset('../data/pd_gy','../data/pd_gy/train.json') #  './split/train.txt'

trdl  = torch.utils.data.DataLoader(trdt, batch_size=5)

In [2]:
trit = iter(trdl)
x,y = next(trit)

In [3]:
assert list(x.shape) == [5, 4, 151, 342]

In [7]:
from torch.utils.data import DataLoader
from dataset import VitalDaset
trdt  = VitalDataset('../data/pd_gy','../data/pd_gy/train.json')
valdt = VitalDataset('../data/pd_gy','../data/pd_gy/val.json') 
tedt  = VitalDataset('../data/pd_gy','../data/pd_gy/test.json') 

trdl  = torch.utils.data.DataLoader(trdt,4)
valdl  = torch.utils.data.DataLoader(valdt,4)
tedl  = torch.utils.data.DataLoader(valdt,4)